<a href="https://colab.research.google.com/github/Tranzil/ProjetoETLPython/blob/master/ProjetoPythonETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Defnições

Este projeto visa a carga de dados sobre interações em chamados de suporte para geração de insights sobre a experiência do usuário. Em linhas gerais, trará a temperatura da interação que o cliente enviou como feedback ao seu atendimento.


## Gerando os registros (Etapa de Extract)

Gerei os registros dos feedbacks dos clientes a partir do seguinte prompt passado para o microsoft copilot em seu modo deepthink.

```python
Poderia gerar uma lista de interações de email, seguindo o formato json abaixo? users = [ {'id': 1, 'name': 'Naruto', 'msg':}, {'id': 2, 'name': 'Hinata', 'msg': } ]
Gere 20 registros, com ids diferentes, com nomes variados, não necessariamente sendo de animes. Nos campos msg, traga interações que sejam um feedback sobre um atendimento de um analista de suporte. Inclua variabilidade nos registros das mensagens, quero que vão desde mensagens amistosas até mensagens de insatisfação. Se possível, inclua detalhamento de situações que o cliente possa ter passado.

```
Como saída, tive o seguinte JSON.

```python

users = [
    {'id': 1, 'name': 'Mariana', 'msg': 'Quero agradecer ao analista João pelo atendimento rápido e claro. Ele fez acesso remoto, identificou o problema de permissão e restaurou meu acesso em 20 minutos. Muito educado e explicou cada passo.'},
    {'id': 2, 'name': 'Carlos', 'msg': 'Atendimento ok, o analista foi cordial, mas precisei abrir o chamado novamente no dia seguinte porque a solução foi temporária. Seria bom um acompanhamento mais efetivo.'},
    {'id': 3, 'name': 'Aline', 'msg': 'Fiquei muito satisfeita: o analista Paula não só resolveu o bug no sistema como também me orientou sobre como evitar que aconteça novamente. Excelente postura profissional.'},
    {'id': 4, 'name': 'Rafael', 'msg': 'Atendimento demorado. Esperei mais de 2 horas na fila e quando fui atendido o analista não tinha acesso às ferramentas necessárias. Acabei resolvendo por conta própria.'},
    {'id': 5, 'name': 'Beatriz', 'msg': 'O suporte foi atencioso, mas a solução exigiu várias trocas de e-mail e ficou confuso. Sugiro que centralizem as informações em um único contato para agilizar.'},
    {'id': 6, 'name': 'Lucas', 'msg': 'Analista muito prestativo (Mateus). Fez diagnóstico remoto, aplicou patch e testou comigo. Voltarei a usar o serviço sem dúvidas.'},
    {'id': 7, 'name': 'Fernanda', 'msg': 'Não gostei do tom do atendimento; o analista foi ríspido e não explicou as etapas. Saí com dúvidas e sem confiança de que o problema foi totalmente resolvido.'},
    {'id': 8, 'name': 'Thiago', 'msg': 'Problema de cobrança resolvido com rapidez. O analista verificou o histórico, corrigiu o lançamento indevido e enviou comprovante. Atendimento eficiente.'},
    {'id': 9, 'name': 'Sofia', 'msg': 'Agradeço ao suporte técnico pela paciência. Tive dificuldades em descrever o erro, mas o analista guiou passo a passo até encontrarmos a causa: conflito de extensão no navegador.'},
    {'id': 10, 'name': 'Marcos', 'msg': 'O técnico prometeu retorno em 24h e não cumpriu. Precisei ligar novamente e perdi tempo. Esperava mais responsabilidade com prazos.'},
    {'id': 11, 'name': 'Patrícia', 'msg': 'Atendimento exemplar: o analista fez testes, documentou a solução e ainda enviou um resumo por e-mail com recomendações para evitar recorrência.'},
    {'id': 12, 'name': 'Eduardo', 'msg': 'Solução encontrada, porém houve perda parcial de dados durante o procedimento de restauração. Preciso de suporte adicional para recuperar arquivos antigos.'},
    {'id': 13, 'name': 'Camila', 'msg': 'O analista foi muito solícito e explicou alternativas. Mesmo não sendo um problema crítico, senti que houve cuidado e profissionalismo no atendimento.'},
    {'id': 14, 'name': 'Gustavo', 'msg': 'Atendimento técnico competente, mas a comunicação entre setores falhou: fui transferido 3 vezes e tive que repetir todo o histórico. Isso poderia ser melhorado.'},
    {'id': 15, 'name': 'Larissa', 'msg': 'Fiquei frustrada porque o analista encerrou o chamado sem confirmar se eu estava satisfeita. A solução funcionou, mas o follow-up é importante.'},
    {'id': 16, 'name': 'Henrique', 'msg': 'Excelente suporte remoto. O analista configurou meu e-mail corporativo corretamente e ainda me mostrou como sincronizar no celular. Muito prático.'},
    {'id': 17, 'name': 'Isabela', 'msg': 'Tive uma experiência negativa: o técnico aplicou uma correção que causou instabilidade no sistema e precisei abrir novo chamado. Solicito revisão do procedimento aplicado.'},
    {'id': 18, 'name': 'Felipe', 'msg': 'Atendimento rápido e objetivo. O analista identificou que o problema era na rede local e orientou o provedor. Resolvido em poucas horas.'},
    {'id': 19, 'name': 'Renata', 'msg': 'Agradeço a atenção, mas o tempo total do atendimento foi longo (mais de 3 horas) devido a testes repetidos. Entendo a complexidade, mas gostaria de mais transparência sobre o progresso.'},
    {'id': 20, 'name': 'Diego', 'msg': 'Muito insatisfeito: o suporte remoto pediu acesso ao meu computador, fez alterações e não documentou nada. Depois disso, perdi configurações importantes e tive prejuízo no trabalho.'}
]

```
### 🟡 Opção 2: Mais Completa (Leitura de Arquivo)

Você pode adaptar o arquivo `SDW2023.csv`, incluindo a coluna `name`, e usá-lo como fonte de dados.

```python
import pandas as pd

# Lê o CSV e converte para uma lista de dicionários
users = pd.read_csv('PythonETL.csv').to_dict(orient='records')

# Garante a estrutura esperada para a etapa de Transformação
for user in users:
    user['msg'] = []
```

## **Extract**

Essa etapa faz a carga do arquivo csv e define o tom da mensagem através da API do OpenAI

In [ ]:
import pandas as pd
import requests
import json
import time

API_KEY = "PASSKEY"
URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# Carrega o dataframe
df = pd.read_csv('/content/csv_data/PythonETL.csv')  # ou já use seu df existente

def classificar_sentimento(texto):
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": (
                    "Você é um classificador de sentimento. "
                    "Classifique a mensagem do cliente em uma das três categorias: "
                    "1) cliente_detrator, 2) cliente_precisa_atencao, 3) cliente_satisfeito. "
                    "Responda apenas com uma das três palavras exatas."
                )
            },
            {"role": "user", "content": texto}
        ]
    }

    response = requests.post(URL, headers=headers, json=payload)

    if not response.ok:
        print(f"API Request failed with status code: {response.status_code}")
        print(f"Response body: {response.json()}")
        # You might want to raise an exception or return a default/error value here
        # For now, let's return an indicator that it failed
        return "erro_na_classificacao"

    resposta = response.json()["choices"][0]["message"]["content"].strip().lower()
    return resposta

# Aplica a classificação linha a linha
resultados = []
for msg in df['msg']:
    sentimento = classificar_sentimento(msg)
    resultados.append(sentimento)
    time.sleep(0.3)  # evita excesso de requisições por segundo

df['sentimentos'] = resultados

# Salva o JSON final
df.to_json("dados_classificados.json", orient="records", force_ascii=False, indent=4)
